In [1]:
import glow
from glow import linear_regression_gwas, expand_struct
from pyspark.ml.linalg import DenseMatrix
from pyspark.sql.session import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as F

def spark_session():
    spark = SparkSession.builder\
        .config('spark.jars.packages', 'io.projectglow:glow_2.11:0.5.0')\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .getOrCreate()
    glow.register(spark)
    return spark

spark = spark_session()
glow.register(spark)

In [48]:
phenotypes = [2, 3, 4, 3, 2, 1, 2, 3, 5, 6]
genotypes =  [0, 1, 2, 2, 2, 0, 1, 1, 2, 2]
covariates = [1, 1, 1, 1, 0, 0, 1, 1, 1, 1]
covariates = [
    [1] * (len(covariates) - 50) + [0] * 50,
    covariates
]

In [59]:
order = 'C'
covars = np.array(covariates).T
covars = DenseMatrix(numRows=covars.shape[0], numCols=covars.shape[1], values=covars.ravel(order).tolist(), isTransposed=False)
df = spark.createDataFrame([Row(genotypes=genotypes, phenotypes=phenotypes, covariates=covars)])
df.select(expand_struct(linear_regression_gwas('genotypes', 'phenotypes', 'covariates'))).collect()

[Row(beta=0.04620364230197803, standardError=0.05474315729563003, pValue=0.3994818201238729)]

In [60]:
order = 'F'
covars = np.array(covariates).T
covars = DenseMatrix(numRows=covars.shape[0], numCols=covars.shape[1], values=covars.ravel(order).tolist(), isTransposed=False)
df = spark.createDataFrame([Row(genotypes=genotypes, phenotypes=phenotypes, covariates=covars)])
df.select(expand_struct(linear_regression_gwas('genotypes', 'phenotypes', 'covariates'))).collect()

[Row(beta=0.11579462606730816, standardError=0.07510350236316143, pValue=0.12440220506998934)]

In [55]:
covars = np.array(covariates).T
df = spark.createDataFrame([Row(genotypes=genotypes, phenotypes=phenotypes)])
df.select(expand_struct(linear_regression_gwas('genotypes', 'phenotypes', F.lit(covars)))).collect()

[Row(beta=0.04620364230197803, standardError=0.05474315729563003, pValue=0.3994818201238729)]